In [2]:
!pip install sentence-transformers faiss-cpu langchain pymupdf openai
!pip install transformers
!pip install -U langchain-community
!pip install openai --upgrade
!pip install sentence-transformers faiss-cpu transformers accelerate pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

Import File

In [7]:
from google.colab import files
uploaded = files.upload()


Saving CV_CIRACI_LUCIO.pdf to CV_CIRACI_LUCIO (1).pdf


Load & Chunk the Document

In [4]:
import fitz  # PyMuPDF
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load PDF
def load_pdf(path):
    doc = fitz.open(path)
    return " ".join(page.get_text() for page in doc)

text = load_pdf("CV_CIRACI_LUCIO.pdf")

# Chunk text
chunks = [text[i:i+500] for i in range(0, len(text), 500)]

Embed Chunks & Store in FAISS

In [5]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(chunks, show_progress_bar=True)

index = faiss.IndexFlatL2(embeddings[0].shape[0])
index.add(np.array(embeddings))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Ask Questions & Retrieve Answers

In [6]:
question = "What is the purpose of this document?"
q_embed = embedder.encode([question])
D, I = index.search(np.array(q_embed), k=3)
retrieved = "\n\n".join([chunks[i] for i in I[0]])


Use a Hugging Face LLM to answer

In [7]:
from transformers import pipeline

qa_model = pipeline("text-generation", model="tiiuae/falcon-7b-instruct", device_map="auto")  # or mistralai/Mistral-7B-Instruct-v0.1

prompt = f"""You are an assistant. Use the following context to answer the question.

Context:
{retrieved}

Question:
{question}
Answer:"""

response = qa_model(prompt, max_new_tokens=200)[0]['generated_text']
print(response)


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


You are an assistant. Use the following context to answer the question.

Context:
s implemented with SCRUM methodology. ServiceDesk also used for internal tickets and 
project. 
• 
Creation and maintenance of IT Dashboards to collect information from ServiceDesk (Ticket 
management) KPI definition. 
• 
Create a global DW to have all EU business information (finance - sales - warranty costs ...) in one 
global DDBB (Snowflake). Participate in global meetings - provide implementation ideas and 
teamwork. 
 
 
I consider myself able to contribute to maximum performance in a chal

 
● Participate in meetings with external vendors.  Identify and evaluate different vendors such as 
Snowflake, dbt, Matillion, NIFI, Azure ML, Jupyter 
VISEO 
   
 
 
 
 
            
 
            Jul 2021 - Jun 2022 
Role: Senior Data Engineer  
● Project planification and implementation. Define task and divide it into the team. Some 
project implemented: 
 o GlobalLeaf, The goal is to create a DataWarehouse t